# Cleanup Image Backend - Google Colab Setup

This notebook runs the backend on a free Google Colab GPU (T4) and exposes it via ngrok.

## Quick Start:
1. **Runtime Type**: Ensure you are using a GPU Runtime (Runtime > Change runtime type > T4 GPU).
2. **Ngrok Token**: You need a free account at [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken).

> **⚠️ IMPORTANT**: During installation (Step 3), Colab might ask you to **"Restart session"**. 
> This is **NORMAL**. Click **"Restart session"**, then run the cells again. 
> (The installation cell might show errors about 'dependency resolver' - you can ignore these generally as long as our app works).

In [ ]:
# 1. Clone Repository
!git clone https://github.com/yudilee/cleanup-image.git
%cd cleanup-image/backend

In [ ]:
# 2. Install System Dependencies
!apt-get update && apt-get install -y libgl1-mesa-glx libhl-amdgpu-pro

In [ ]:
# 3. Install Python Dependencies
# NOTE: This may downgrade NumPy and ask for a session restart. 
# If asked, click 'Restart session', then run this cell again (it will be fast) and proceed.
!pip install -r requirements.txt
!pip install pyngrok uvicorn nest_asyncio

In [ ]:
# 4. Install PixelHacker Dependencies (Optional - for high quality model)
# Uncomment the lines below if you want to use the PixelHacker model
# !pip install diffusers==0.30.2 transformers==4.40.0 accelerate==0.34.0 peft

In [ ]:
# 5. Setup Ngrok & Run Server
import getpass
import os
import sys
import subprocess
from pyngrok import ngrok, conf

# Get Ngrok Token
print("Please enter your Ngrok Authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken):")
try:
    token = getpass.getpass('Ngrok Authtoken: ')
except Exception as e:
    token = input("Ngrok Authtoken: ")

if token:
    conf.get_default().auth_token = token
    
    # Kill existing tunnels
    ngrok.kill()

    # Start tunnel
    public_url = ngrok.connect(8000)
    print(f"\n\n🚀 Backend is running! Copy this URL into your Frontend ENV or Code: {public_url}\n\n")
    
    # Ensure we are in the backend directory
    if os.path.basename(os.getcwd()) != 'backend':
        if os.path.exists('cleanup-image/backend'):
            os.chdir('cleanup-image/backend')
        elif os.path.exists('backend'):
            os.chdir('backend')
    
    print(f"Running server in directory: {os.getcwd()}")
    
    # Run Uvicorn as a SUBPROCESS to avoid event loop conflicts in Colab
    # This isolates the server process from the notebook's asyncio loop
    cmd = [sys.executable, "-m", "uvicorn", "main:app", "--host", "127.0.0.1", "--port", "8000"]
    
    try:
        # subprocess.run will block this cell and stream output to the console
        subprocess.run(cmd, check=True)
    except KeyboardInterrupt:
        print("Server stopped by user.")
    except Exception as e:
        print(f"❌ Error running server: {e}")
else:
    print("❌ Error: Ngrok token is required to expose the server.")